In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
% cd /content/drive/My Drive/xview/spacenet/src/models
import sys
#sys.path.append("../../src/models")

from evaluate_model import eval_semantic_segmentation, calc_semantic_segmentation_confusion
from segmentation import SegmentationModel as Model

from os import path
from PIL import Image
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

/content/drive/My Drive/xview/spacenet/src/models


In [ ]:
# Load trained model
# Modify the the paths based on your trained model location if needed.
mean = np.load("/content/drive/My Drive/xview/test-data/spacenet_gt_test/dataset/mean.npy")
model = Model("/content/drive/My Drive/xview/localization.h5", mean)

In [ ]:
# Load test-set
# Modify the the paths based on your data location if needed.
with open( "/content/drive/My Drive/xview/test-data/spacenet_gt_test/dataset/test.txt") as f:
    lines = f.readlines()

test_set = []
for line in lines:
    line = line.rstrip()
    test_set.append(line)

In [ ]:
batchsize=100

enable_cupy = False # No CuPy (Use NumPy)
n_epoch=2 # Only 2 epochs

In [ ]:
import math

def apply_augmentation(image):
    image_in, crop = preprocess(image)

    top, left, bottom, right = crop
    image_in = image_in[:, top:bottom, left:right]
    return image_in

In [ ]:
import os
from chainer import cuda, serializers, Variable
def preprocess(image):
    mean = np.load(os.path.join("/content/drive/My Drive/xview/test-data/spacenet_gt_test/dataset/", "mean.npy"))
    mean = mean[np.newaxis,np.newaxis,:]

    h,w,_ = image.shape
    h_padded = int(math.ceil(float(h)/16.0)*16)
    w_padded = int(math.ceil(float(w)/16.0)*16)

    pad_y1 = (h_padded - h)//2
    pad_x1 = (w_padded- w)//2
    pad_y2 = h_padded - h - pad_y1
    pad_x2 = w_padded - w - pad_x1

    image_padded = np.pad(image, ((pad_y1, pad_y2),(pad_x1, pad_x2),(0,0)),'symmetric')
    image_in = (image_padded - mean)/255.0
    image_in = image_in.transpose(2,0,1)
    #image_in = image_in[np.newaxis, :, :, :]
    #image_in = Variable(cuda.cupy.asarray(image_in, dtype=cuda.cupy.float32))

    top, left = pad_y1, pad_x1
    bottom, right = top+h, left+w
    #print(image_in.shape)
    return image_in, (top,left,bottom,right)

In [ ]:
#  Evaluate model

# Modify the the paths based on your raster image and building label location.
image_dir = '/content/drive/My Drive/xview/test-data/spacenet_gt_test/images'
label_dir = '/content/drive/My Drive/xview/test-data/spacenet_gt_test/labels'

gt_labels = []
pred_labels = []
unet_iou = []

for test_data in tqdm(test_set):
    image_path = path.join(image_dir, test_data)
    label_path = path.join(label_dir, test_data)
    
    image = np.array(Image.open(image_path))
    label = np.array(Image.open(label_path))
    label = apply_augmentation(label)
    label = label[0]
    # Make gt_label, ground-truth building mask
    h, w  = label.shape
    gt_label = np.zeros(shape=[h, w], dtype=np.int32) # 0: background
    gt_label[label > 0] = 1 # 1: "building"
    
    # Make pred_label, predicted building mask
    score = model.apply_segmentation(image)
    pred_label = np.argmax(score, axis=0)
    
    gt_labels.append(gt_label)
    pred_labels.append(pred_label)

100%|██████████| 752/752 [20:19<00:00,  1.62s/it]


In [ ]:
result = eval_semantic_segmentation(pred_labels, gt_labels)
print(result)

{'iou': array([0.98198353, 0.73683993]), 'miou': 0.8594117277168716, 'pixel_accuracy': 0.9828487317612831, 'class_accuracy': array([0.99349779, 0.81317661]), 'mean_class_accuracy': 0.9033372020879059}


In [ ]:
result = eval_semantic_segmentation(pred_labels, gt_labels)
building_class = 1
print("IoU for class Building = ", result['iou'][building_class])
print("Accuracy for class Building = ", result['class_accuracy'][building_class])

IoU for class Building =  0.736839929000374
Accuracy for class Building =  0.8131766129245268


In [ ]:
confusion = calc_semantic_segmentation_confusion(pred_labels, gt_labels)
print("confusion:")
print(confusion)

tn = confusion[0][0]
fp = confusion[0][1]
tp = confusion[1][1]
fn = confusion[1][0]

precision = float(tp) / float(tp + fp)
recall       = float(tp) / float(tp + fn)
test_accuracy = float(tp + tn) / float(tp + tn + fp + fn)
f1_score = 2*((precision * recall) / (precision + recall))
print()
print("accuracy: ", test_accuracy)
print("precision: ", precision)
print("recall: ", recall)
print("f1-score: ",f1_score)

confusion:
[[737137332   4824390]
 [  8699885  37867545]]

accuracy:  0.9828487317612831
precision:  0.8869952837696394
recall:  0.8131766129245268
f1-score:  0.8484834056348037


In [ ]:
import csv
import numpy as np

yourArray = np.array(unet_iou)

with open('/content/drive/My Drive/xview/test-data/spacenet_gt_test/dataset/outputPreviousIouFile.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in range(0,yourArray.shape[0]):
        myList = []
        myList.append(yourArray[row])
        writer.writerow(myList)